In [ ]:
# !pip install mne

In [3]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn import metrics
from mne.decoding import CSP
from scipy.signal import butter, lfilter

## Import Data

In [4]:
data_dir = './data/flatten_individual_trim/'
np_data = np.genfromtxt(data_dir + 'Beam' + '_flatten.csv', delimiter = ',')

In [5]:
n_row = np_data.shape[0]
n_col = np_data.shape[1]
X = np_data[:, np.arange(0, n_col - 1, 1)]
y = np_data[:, -1]

In [6]:
def butter_bandpass(lowcut, highcut, fs, order = 5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype = 'band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order = 5):
    b, a = butter_bandpass(lowcut, highcut, fs, order = order)
    y = lfilter(b = b, a = a, x = data, axis = 1)
    return y

In [7]:
X_new = butter_bandpass_filter(X, 0.1, 30., 1024, order = 5)

In [8]:
X_new

array([[-7.66604589e-006, -7.96311003e-005, -4.12769496e-004, ...,
         7.25249665e+106,  7.23408333e+106,  7.21500223e+106],
       [ 2.84383695e-006,  2.95696046e-005,  1.53446330e-004, ...,
        -1.32545759e+107, -1.32386044e+107, -1.32216173e+107],
       [ 2.82045883e-006,  2.93247759e-005,  1.52216378e-004, ...,
        -7.97339928e+106, -8.02275356e+106, -8.07222091e+106],
       ...,
       [-2.46868128e-006, -2.56690109e-005, -1.33227534e-004, ...,
         1.69960616e+107,  1.69273210e+107,  1.68565418e+107],
       [ 5.85274455e-006,  6.09036916e-005,  3.16311242e-004, ...,
        -1.26216063e+107, -1.26553644e+107, -1.26887444e+107],
       [-1.46635794e-006, -1.53822093e-005, -8.06855421e-005, ...,
        -1.17698394e+107, -1.18910806e+107, -1.20131216e+107]])

In [ ]:
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components = 4, reg = None, log = True, norm_trace = False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('LDA', lda)])

# clf = LinearDiscriminantAnalysis(n_components = None)
# clf.fit(X, y)
predicted = cross_val_predict(clf, X, y, cv = 10)

/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [5]:
metrics.accuracy_score(y, predicted) 

0.78097982708933722

In [ ]:
metrics.f1_score(y, predicted)

In [ ]:
metrics.confusion_matrix(y, predicted)

In [ ]:
tn, fp, fn, tp = metrics.confusion_matrix(y, predicted).ravel()

## True Positive Rate (TPR) or Sensitivity
$TPR=\frac{TP}{P}=\frac{TP}{TP+FN}$

Ref: https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [ ]:
tpr = tp / (tp + fn)
print(tpr)

## False Positive Rate (FPR) or Fall-out
$FPR=\frac{FP}{N}=\frac{FP}{FP+TN}$

In [ ]:
fpr = fp / (fp + tn)
print(fpr)